# Holy 🦆uck! Fast Analysis with DuckDB + Pyarrow
> Trying out some new speedy tools for data analysis

- toc: true 
- badges: true
- categories: [python, data, intermediate]

# Holy 🦆uck! Fast Analysis with DuckDB + Pyarrow

Turning to DuckDB when you need to crunch more numbers faster than pandas in your Streamlit app 🎈

Inspired by "DuckDB quacks Arrow" blogpost cross-posted on [duckdb](https://duckdb.org/2021/12/03/duck-arrow.html) and [arrow](https://arrow.apache.org/blog/2021/12/03/arrow-duckdb/)

## Background

`streamlit` and Streamlit Cloud are fantastic for sharing your data exploration apps.
A very common pattern uses csv files with `pandas` to accomplish the necessary steps of:

- Load the data into the program
- Filter data by certain columns or attributes
- Compute analyses on the data (averages, counts, etc.)

## NYC Uber Data

Let's take this NYC Uber dataset example from Streamlit.
We'll pay attention to:

- How much RAM / memory is used
- How long it takes to perform each step


In [170]:
import pandas as pd
import numpy as np
# import streamlit as st

# singleton ignored because we're not in streamlit anymore
# @st.experimental_singleton
def load_data():
    data = pd.read_csv(
        "uber-raw-data-sep14.csv.gz",
        nrows=100000,  # approx. 10% of data
        names=[
            "date/time",
            "lat",
            "lon",
        ],  # specify names directly since they don't change
        skiprows=1,  # don't read header since names specified directly
        usecols=[0, 1, 2],  # doesn't load last column, constant value "B02512"
        parse_dates=[
            "date/time"
        ],  # set as datetime instead of converting after the fact
    )

    return data

In [171]:
%%time
data = load_data()

CPU times: user 3.29 s, sys: 33 ms, total: 3.33 s
Wall time: 3.32 s


In [172]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   date/time  100000 non-null  datetime64[ns]
 1   lat        100000 non-null  float64       
 2   lon        100000 non-null  float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 2.3 MB


Feel free to reference the `read_csv` [documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html), the focus of this post is on the `nrows=100000` argument though.

This `nrows` is used to limit the number of rows that get loaded into our application.
Taking in `100,000` rows landed us around `2.3 MB` of memory allocation for the data.

It loaded on my computer in `~3` seconds.

Let's see how that would go without our `nrows` limitation

In [173]:
def load_full_data():
    data = pd.read_csv(
        "uber-raw-data-sep14.csv.gz",
        # nrows=100000,  # approx. 10% of data
        names=[
            "date/time",
            "lat",
            "lon",
        ],  # specify names directly since they don't change
        skiprows=1,  # don't read header since names specified directly
        usecols=[0, 1, 2],  # doesn't load last column, constant value "B02512"
        parse_dates=[
            "date/time"
        ],  # set as datetime instead of converting after the fact
    )

    return data

In [174]:
%%time
full_data = load_full_data()

CPU times: user 33.6 s, sys: 243 ms, total: 33.8 s
Wall time: 33.7 s


In [175]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028136 entries, 0 to 1028135
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   date/time  1028136 non-null  datetime64[ns]
 1   lat        1028136 non-null  float64       
 2   lon        1028136 non-null  float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 23.5 MB


Ok, so with `~10` times as much data (`1,028,136` vs `100,000`) we use:

- `~10` times as much memory (`23.5 MB` vs `2.3 MB`)
- `~10` times as much time (`30.1 s` vs `2.99 s`)

The first time this app loads in `streamlit` will be a bit slow either way, but the `singleton` decorator is designed to prevent having to re-compute objects like this.

(Also note that this is a single month of data... a year might include `~12,337,632` entries based on this september 2014 data)

## Enter the Duck

Using `pyarrow` and `duckdb` let's see if we get any improvement

In [176]:
import duckdb
import pyarrow as pa
from pyarrow import csv
import pyarrow.dataset as ds

def load_data_duckdb():
    data = csv.read_csv('uber-raw-data-sep14.csv.gz', convert_options=csv.ConvertOptions(
        include_columns=["Date/Time","Lat","Lon"],
        timestamp_parsers=['%m/%d/%Y %H:%M:%S']
    )).rename_columns(['date/time', 'lat', 'lon'])

    # `dataset` is for partitioning larger datasets. Can't include timestamp parsing directly though
    # data = ds.dataset("uber-raw-data-sep14.csv.gz", schema=pa.schema([
    #     ("Date/Time", pa.timestamp('s')),
    #     ('Lat', pa.float32()),
    #     ('Lon', pa.float32())
    # ]), format='csv')

    # DuckDB can query Arrow tables, so we'll just return the table and a connection for flexible querying
    return data, duckdb.connect(":memory:")

In [177]:
%%timeit
arrow_data, con = load_data_duckdb()

121 ms ± 4.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [178]:
arrow_data[:5]

pyarrow.Table
date/time: timestamp[s]
lat: double
lon: double
----
date/time: [[2014-09-01 00:01:00,2014-09-01 00:01:00,2014-09-01 00:03:00,2014-09-01 00:06:00,2014-09-01 00:11:00]]
lat: [[40.2201,40.75,40.7559,40.745,40.8145]]
lon: [[-74.0021,-74.0027,-73.9864,-73.9889,-73.9444]]

Holy Smokes! Well that was fast and fun!

`pyarrow` read the whole dataset in `120 ms`.
That's `0.120 s` compared to `30.1 s` with `pandas`!

So how much memory are `pyarrow` and `duckdb` using?

In [179]:
def format_bytes(size):
    """from https://stackoverflow.com/a/49361727/15685218"""
    # 2**10 = 1024
    power = 2**10
    n = 0
    power_labels = {0 : '', 1: 'kilo', 2: 'mega', 3: 'giga', 4: 'tera'}
    while size > power:
        size /= power
        n += 1
    return size, power_labels[n]+'bytes'

In [180]:
format_bytes(arrow_data.nbytes)

(23.53216552734375, 'megabytes')

Ok, the `pyarrow` table has roughly the same size as the full `pandas` Dataframe

In [181]:
con.execute('PRAGMA database_size;')
"""
database_size VARCHAR, -- total block count times the block size
block_size BIGINT,     -- database block size
total_blocks BIGINT,   -- total blocks in the database
used_blocks BIGINT,    -- used blocks in the database
free_blocks BIGINT,    -- free blocks in the database
wal_size VARCHAR,      -- write ahead log size
memory_usage VARCHAR,  -- memory used by the database buffer manager
memory_limit VARCHAR   -- maximum memory allowed for the database
"""
database_size, block_size, total_blocks, used_blocks, free_blocks, wal_size, memory_usage, memory_limit = con.fetchall()[0]
memory_usage

'0 bytes'

We haven't told `duckdb` to load anything into its own tables, so it still has no memory usage.
Nevertheless, `duckdb` can query the `arrow_data` since it's a `pyarrow` table.
(`duckdb` can also load directly [from csv](https://duckdb.org/docs/data/csv)).

So where does that leave us on loading the full `1,000,000` row dataset?

- `pandas`: `~30 s` of time and `23.5 MB`
- `pyarrow`: `~.1 s` of time (`120 ms`) and `23.9 MB`

In fairness, I tried `pandas` with the `pyarrow` engine.
At the time of writing I can't find a fast datetime parse and `usecols` throws an error in `pyarrow` (see end of post).
Reading the full CSV without datetime parsing is in line in terms of speed though.

(also see why the best CSV [is not a CSV at all](https://pythonspeed.com/articles/pandas-read-csv-fast/) for more on this path)

In [182]:
%%time
arrow_df = pd.read_csv(
    "uber-raw-data-sep14.csv.gz",
    engine='pyarrow',
    names=[
        "date/time",
        "lat",
        "lon",
        "CONST"
    ],  # specify names directly since they don't change
    skiprows=1,  # don't read header since names specified directly
    # usecols=[1, 2],  # doesn't load last column, constant value "B02512"
    parse_dates=[
        "date/time"
    ],  # set as datetime instead of converting after the fact
    # infer_datetime_format=True  # Unsupported for pyarrow
    date_parser=lambda x: pd.to_datetime(x)
)

CPU times: user 33.6 s, sys: 248 ms, total: 33.9 s
Wall time: 33.7 s


In [183]:
arrow_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1028136 entries, 0 to 1028135
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   date/time  1028136 non-null  datetime64[ns]
 1   lat        1028136 non-null  float64       
 2   lon        1028136 non-null  float64       
 3   CONST      1028136 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 31.4+ MB


In [184]:
%%timeit
arrow_df_no_datetime = pd.read_csv(
    "uber-raw-data-sep14.csv.gz",
    engine='pyarrow',
    names=[
        "date/time",
        "lat",
        "lon",
        "CONST"
    ],  # specify names directly since they don't change
    skiprows=1,  # don't read header since names specified directly
    # usecols=[1, 2],  # doesn't load last column, constant value "B02512"
)

137 ms ± 189 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Filtration

We have 3 main analysis functions to compare between `pandas` and `duckdb` for this app, laid out below:

In [185]:
# FILTER DATA FOR A SPECIFIC HOUR, CACHE
# @st.experimental_memo
def filterdata(df, hour_selected):
    return df[df["date/time"].dt.hour == hour_selected]


# CALCULATE MIDPOINT FOR GIVEN SET OF DATA
# @st.experimental_memo
def mpoint(lat, lon):
    return (np.average(lat), np.average(lon))


# FILTER DATA BY HOUR
# @st.experimental_memo
def histdata(df, hr):
    filtered = data[
        (df["date/time"].dt.hour >= hr) & (df["date/time"].dt.hour < (hr + 1))
    ]

    hist = np.histogram(filtered["date/time"].dt.minute, bins=60, range=(0, 60))[0]

    return pd.DataFrame({"minute": range(60), "pickups": hist})

In [186]:
%%timeit
# For fairness, we'll use the full dataframe
filterdata(full_data, 14)

17.8 ms ± 116 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [187]:
%%timeit
mpoint(full_data["lat"], full_data["lon"])

404 µs ± 564 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [188]:
%%timeit
histdata(full_data, 14)

/var/folders/cp/ktx4zddx7q3bqctqfjykn5700000gn/T/ipykernel_82767/2026438809.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = data[


39.5 ms ± 275 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


How about Duckdb (with conversion back to `pandas` for fairness)

In [189]:
def duck_filterdata(con, hour_selected):
    return con.query(
        f'SELECT "date/time", lat, lon FROM arrow_data WHERE hour("date/time") = {hour_selected}'
    ).to_df()


def duck_mpoint(con):
    return con.query("SELECT AVG(lat), AVG(lon) FROM arrow_data").fetchone()


def duck_histdata(con, hr):
    hist_query = f'SELECT histogram(minute("date/time")) FROM arrow_data WHERE hour("date/time") >= {hr} and hour("date/time") < {hr + 1}'
    results, *_ = con.query(hist_query).fetchone()
    return pd.DataFrame(results)

In [190]:
%%timeit
duck_filterdata(con, 14)

6.25 ms ± 20.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [191]:
%%timeit
duck_mpoint(con)

1.64 ms ± 8.87 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [192]:
%%timeit
duck_histdata(con, 14)

2.64 ms ± 26.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



We got a modest improvement in `filterdata` and more than 10x speedup in `histdata`, but actually lost out to `numpy` for finding the average of 2 arrays in `mpoint`!

- `filterdata`:
  - `pandas`: 19.1 ms ± 284 µs
  - `duckdb`: 6.53 ms ± 126 µs
- `mpoint`:
  - `numpy`: 403 µs ± 5.35 µs
  - `duckdb`: 1.7 ms ± 82.6 µs
- `histdata`:
  - `pandas` + `numpy`: 40.8 ms ± 430 µs
  - `duckdb`: 2.93 ms ± 28.4 µs

In [193]:
19.1 / 6.53

2.9249617151607965

In [194]:
403 / 1700

0.23705882352941177

In [195]:
40.8 / 2.93

13.924914675767916

## Conclusions

It's no secret that Python is not a fast language, but there are tricks to speed it up.
Common advice is to utilize C optimizations via `numpy` and `pandas`.

Another new contender is utilizing the C++ driven `duckdb` as an in-process OLAP database manager.
It takes some re-writing of Python code into SQL, but can play nicely with `pandas` and `pyarrow`.

Speaking of Arrow, it seems to be 

This post explores an example `streamlit` app that utilizes some `pandas` and `numpy` functions such as `read_csv`, `average`, and DataFrame slicing

In [196]:
pd.read_csv(
    "uber-raw-data-sep14.csv.gz",
    # nrows=100000,  # approx. 10% of data
    engine='pyarrow',
    names=[
        "date/time",
        "lat",
        "lon",
        # "CONST"
    ],  # specify names directly since they don't change
    skiprows=1,  # don't read header since names specified directly
    # usecols=[1, 2],  # doesn't load last column, constant value "B02512"
    parse_dates=[
        "date/time"
    ],  # set as datetime instead of converting after the fact
    # # infer_datetime_format=True  # Unsupported for pyarrow
    date_parser=lambda x: pd.to_datetime(x)
)

,0,date/time,lat,lon
0,9/1/2014 0:01:00,1970-01-01 00:00:00.000000040,-74.0021,B02512
1,9/1/2014 0:01:00,1970-01-01 00:00:00.000000040,-74.0027,B02512
2,9/1/2014 0:03:00,1970-01-01 00:00:00.000000040,-73.9864,B02512
3,9/1/2014 0:06:00,1970-01-01 00:00:00.000000040,-73.9889,B02512
4,9/1/2014 0:11:00,1970-01-01 00:00:00.000000040,-73.9444,B02512
...,...,...,...,...
1028131,9/30/2014 22:57:00,1970-01-01 00:00:00.000000040,-73.9845,B02764
1028132,9/30/2014 22:57:00,1970-01-01 00:00:00.000000040,-74.1773,B02764
1028133,9/30/2014 22:58:00,1970-01-01 00:00:00.000000040,-73.9319,B02764
1028134,9/30/2014 22:58:00,1970-01-01 00:00:00.000000040,-74.0066,B02764


In [197]:
pd.read_csv(
    "uber-raw-data-sep14.csv.gz",
    # nrows=100000,  # approx. 10% of data
    engine='pyarrow',
    # names=[
    #     "date/time",
    #     "lat",
    #     "lon",
    #     "CONST"
    # ],  # specify names directly since they don't change
    # skiprows=1,  # don't read header since names specified directly
    usecols=[0,1],  # doesn't load last column, constant value "B02512"
    # parse_dates=[
    #     "date/time"
    # ],  # set as datetime instead of converting after the fact
    # # infer_datetime_format=True  # Unsupported for pyarrow
    # date_parser=lambda x: pd.to_datetime(x)
).info()

TypeError: expected bytes, int found